# Wordler Game Engine

## Initial Imports

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, HTML
from enum import auto, Enum
import getpass

# Wordler utilities

In [2]:
class Wordler():
#     Static stuff (i.e. instance independent)
    @staticmethod
    def get_english_words_path():
        this_dir_path = os.path.dirname(os.path.realpath('__file__'))
#         print (this_dir_path) 
        # These are all the english words
        english_words_path = os.path.abspath(os.path.join(this_dir_path, 'english-words'))
        return english_words_path

    @staticmethod
    def reset_word_list(word_length: int = 5):
        english_words_path = Wordler.get_english_words_path()
        words_alpha_list = pd.read_csv(os.path.join(english_words_path, 'words_alpha.txt'), header=None, names=['Words'], dtype='str')
    #     display(words_alpha_list)
        mask = (words_alpha_list.Words.str.len() == word_length)
        wordle_valid_list = words_alpha_list[mask]
        wordle_valid_list.to_csv(os.path.join(english_words_path, 'valid_wordles.txt'), header=None, index=None)

    @staticmethod
    def get_word_list():
        english_words_path = Wordler.get_english_words_path()
        wordle_valid_list = pd.read_csv(os.path.join(english_words_path, 'valid_wordles.txt'), header=None, names=['Words'], dtype='str').Words.values
#         display(wordle_valid_list)
        return wordle_valid_list

# Initialise
    def __init__(self, word_length: int = 5):
        self.word_length = 5
        if not(word_length is None):
            if word_length != 5:
                Wordler.reset_word_list(word_length)
                self.word_length = word_length                
        self.valid_words = Wordler.get_word_list()
    def is_valid_wordle(self, word:str):
        word = str.lower(word)
        is_valid_wordle = word in self.valid_words
        return is_valid_wordle

    class Wordle_Status(Enum):
        PLAYING = auto()
        WON = auto()
        LOST = auto()
        
    class Guess_Result(Enum):
        NotInWord = auto()
        InCorrectPlace = auto()
        CorrectPlace = auto()
        
    


## Wordler Game

In [3]:
# Gaming stuff
class Game():
    class Guess_Result():
        def __init__(self, 
                     msg = '', 
                     status: Wordler.Wordle_Status = Wordler.Wordle_Status.PLAYING, 
                     tries_left: int = 6, 
                     results: dict = dict(),
                     guesses: list = []
                    ):
            self.msg = msg
            self.results = results
            self.status = status
            self.tries_left = tries_left

    def __init__(self,word_to_guess: str):
        self.engine = Wordler() # Initialises and caches valid words
        word_to_guess = str.lower(word_to_guess)
        is_valid_wordle = self.engine.is_valid_wordle(word_to_guess)
        if is_valid_wordle:
            print(f'Good start buddy. That word is indeed a valid Wordle. ;-)')
        else:
             raise Exception(f'Come on!!!! Jeez! "{word_to_guess}" is not a valid Wordle.')   
        self.word_to_guess = word_to_guess
        self.max_tries = 6
        self.tries_left = self.max_tries
        self.status = Wordler.Wordle_Status.PLAYING
        self.guesses = []
        self.results_per_guess = []

    def tries_left(self):
        return self.tries_left

    def status(self):
        return self.status

    def guesses_msg(self):
        tries_taken = self.max_tries - self.tries_left
        msg = ''
        msg += 'Your guesses have been: ' + ','.join(self.guesses) + '\n'
        msg += f'You have used up {tries_taken}/{self.max_tries} tries.\n'
        return msg
    
    def printmd(results:list = []):
        def __print_inside_box(c: str, color: str):
            md_string = f" <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: {color};'>{c}</span> "
            return md_string
        md_string = '<p>'
        for result in results:
            letter = result[0]
            result_type = result[1]
            if result_type == Wordler.Guess_Result.CorrectPlace:
                md_string +=  __print_inside_box(letter, 'green')
            elif result_type == Wordler.Guess_Result.InCorrectPlace:
                md_string +=  __print_inside_box(letter, 'orange')
            else:
                md_string +=  __print_inside_box(letter, 'red')
        md_string += '</p>'
#         print(md_string)
        display(Markdown(md_string))

    
    def won_result(self, results:list = []):
        tries_taken = self.max_tries - self.tries_left
        guesses = ','.join(self.guesses)
        msg = f'You already won in {tries_taken}/{self.max_tries} tries.\n'
        msg += 'Congratulations.' + self.guesses_msg() + '\n'
        msg += f'Wordle was {self.word_to_guess}.'
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            results = results,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result

    def lost_result(self, results:list = []):
        tries_taken = self.max_tries - self.tries_left
        guesses = ','.join(self.guesses)
        msg = f'You already lost in {tries_taken}.\n'
        msg += 'Keep playing.' + self.guesses_msg() + '\n'
        msg += f'Wordle was {self.word_to_guess}.'
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            results = results,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result

    def invalid_guess_result(self, msg: str = ''):
        tries_taken = self.max_tries - self.tries_left
        guesses = ','.join(self.guesses)
        msg = f'Invalid Entry because: {msg}' + '\n'
        msg += 'Keep playing.' + self.guesses_msg()          
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result


    def playing_result(self, results:list = []):
        tries_taken = self.max_tries - self.tries_left
        msg = 'Keep playing.' + self.guesses_msg()
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            results = results,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result

    def guess(self, word:str):
        word = str.lower(word)
        if (self.status == Wordler.Wordle_Status.WON):
            return self.won_result()
        if (self.status == Wordler.Wordle_Status.LOST):
            return self.lost_result()
        if (word in self.guesses):
            return self.invalid_guess_result(f'{word} is in previous guesses.')
        is_valid_wordle = self.engine.is_valid_wordle(word)
        if not (is_valid_wordle):
            return self.invalid_guess_result(f'{word} is not in list of words. Please try again.')
        else:
            self.tries_left -= 1
            results = []
            correct_guesses = 0
            self.guesses.append(word)
            for i in range(self.engine.word_length):
                if (word[i] == self.word_to_guess[i]):
                    this_result = Wordler.Guess_Result.CorrectPlace
                    correct_guesses += 1
                elif (word[i] in self.word_to_guess): 
                    this_result = Wordler.Guess_Result.InCorrectPlace
                else:
                    this_result = Wordler.Guess_Result.NotInWord
                results.append((word[i], this_result))
            Game.printmd(results)
#                 print(f'{word[i]} : {this_result}')


            self.results_per_guess.append(results)
            if (correct_guesses == self.engine.word_length):
                self.status = Wordler.Wordle_Status.WON
                return self.won_result(results=results)
            else:
                if(self.tries_left == 0):
                    self.status = Wordler.Wordle_Status.LOST
                    return self.lost_result(results=results)
                else:
                    return self.playing_result(results=results)
    def play_manual(self):
        color_key = [
            ('In the correct place is green', Wordler.Guess_Result.CorrectPlace),
            ('In the word, but not in the correct place is orange', Wordler.Guess_Result.InCorrectPlace),
            ('Not in the word is red', Wordler.Guess_Result.NotInWord),
        ]
        Game.printmd(color_key)
        def is_number(guess):
            try:
                # Convert it into integer
                val = int(guess)
                return True
            except ValueError:
                try:
                    # Convert it into float
                    val = float(guess)
                    return True
                except ValueError:
                    return False

        while self.status == Wordler.Wordle_Status.PLAYING:
            which_try = self.max_tries - self.tries_left + 1
            this_guess = input(f'Enter your guess #{which_try} (enter number to quit the game): ')
            if is_number(this_guess):
                print('Sorry to see you go. Lets play another time.')
                break
            self.guess(this_guess)





## Create your own wordle game without revealing the word

In [4]:
def create_wordler():
    msg = 'Welcome WORDLE host. What is the word that people need to guess? '
    word_to_guess = getpass.getpass(msg)
    game = Game(word_to_guess)
    return game

## Example of how to create and play

In [5]:
game = create_wordler()

Welcome WORDLE host. What is the word that people need to guess?  ·····


Good start buddy. That word is indeed a valid Wordle. ;-)


In [6]:
game.play_manual()

<p> <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: green;'>In the correct place is green</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: orange;'>In the word, but not in the correct place is orange</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: red;'>Not in the word is red</span> </p>

Enter your guess #1 (enter number to quit the game):  stair


<p> <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: orange;'>s</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: orange;'>t</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: green;'>a</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: red;'>i</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: red;'>r</span> </p>

__________________________________________________________________________
Keep playing.Your guesses have been: stair
You have used up 1/6 tries.

__________________________________________________________________________


Enter your guess #2 (enter number to quit the game):  ucorn


...

Enter your guess #3 (enter number to quit the game):  plast


__________________________________________________________________________
Invalid Entry because: plast is not in list of words. Please try again.
Keep playing.Your guesses have been: stair,plume
You have used up 2/6 tries.

__________________________________________________________________________


Enter your guess #3 (enter number to quit the game):  blast


<p> <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: green;'>b</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: green;'>l</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: green;'>a</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: green;'>s</span>  <span style='font-size:30px; background:#F0FFFF; border:3px; border-style:solid; border-color:#FF0000; padding: 1em; overflow: visible; color: green;'>t</span> </p>

__________________________________________________________________________
You already won in 3/6 tries.
Congratulations.Your guesses have been: stair,plume,blast
You have used up 3/6 tries.

Wordle was blast.
__________________________________________________________________________
